In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import json


dataframes = ["TheftEvent.parquet"]
attach_config_vars = ["generation_empty_w"]

folder = Path(".").resolve() / "data" / "experiments"
regex = r"(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})_(.+)_(\d{5})"


def load_experiments_by_name(name) -> dict[str, pd.DataFrame]:

    dfs = {dfname: [] for dfname in dataframes}

    for ent in folder.iterdir():

        if not ent.is_dir():
            continue

        res = re.match(regex, ent.name)
        
        if res is None:
            continue

        dstr, namestr, randstr = res.group(1), res.group(2), res.group(3)

        if namestr != name:
            continue

        if not (ent / "config.json").is_file():
            continue

        with open(ent / "config.json", "r", encoding="utf-8") as f:
            config = json.load(f)
        
        skip_log = not all([(ent / dfname).is_file() for dfname in dataframes])
    
        if skip_log:
            continue

        for dfname in dataframes:
            df = pd.read_parquet(ent / dfname) 
            df["rand"]= randstr
            
            for av in attach_config_vars:
                df[av] = config[av]
            
            dfs[dfname].append(df)

        

    
    dfs = {dfname: pd.concat(inners, ignore_index=True) for dfname, inners in dfs.items()}

    return dfs
"""
Available events:
 - TheftEvent.parquet
"""
df = load_experiments_by_name("vision_impr_thieves")["TheftEvent.parquet"]